---
title: A Dynamic Changepoint Model for New Product Sales Forecasting
author: Abdullah Mahmood
date: last-modified
format:
    html:
        theme: cosmo
        css: quarto-style/style.css
        highlight-style: atom-one
        mainfont: Palatino
        fontcolor: black
        monobackgroundcolor: white
        monofont: Menlo, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New, monospace
        fontsize: 13pt
        linestretch: 1.4
        number-sections: true
        number-depth: 5
        toc: true
        toc-location: right
        toc-depth: 5
        code-fold: true
        code-copy: true
        cap-location: bottom
        format-links: false
        embed-resources: true
        anchor-sections: true
        code-links:
        -   text: GitHub Repo
            icon: github
            href: https://github.com/abdullahau/customer-analytics/
        -   text: Quarto Markdown
            icon: file-code
            href: https://github.com/abdullahau/customer-analytics/blob/main/New-Product-Sales-Forecasting-Dynamic-Changepoint-Model.qmd
        html-math-method:
            method: mathjax
            url: https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js
jupyter: python3
---

This models presented in this notebook are a direct implementation of the modes proposed by *Peter S. Fader, Bruce G. S. Hardie, and Chun-Yao Huang* in their 2004 paper [*A Dynamic Changepoint Model for New Product Sales Forecasting*](https://www.jstor.org/stable/30036656) and *Sunil Gupta's* 1991 paper  [*Stochastic Models of Interpurchase Time with Time-Dependent Covariates*](https://journals.sagepub.com/doi/10.1177/002224379102800101)

The models discussed here can generally be classified as **stochastic models of interpurchase time**. These models are used extensively in marketing literature for new product sales-forecasting for consumer packaged goods and usually fit and predict data quite well. 

There are four stochastic models typically used in a marketing context: *exponential* and *Erlang-2* at the *individual-consumer level* and these two distribution with *gamma heterogeneity* to model *aggregate-level behavior*. In his 1991 paper, Sunil Gupta proposed a variant of this model to include *marketing mix covariates* for better predictive and diagnostic purposes. Fader et al followed up on Gupta's model by nesting a number of **multiple-event timing processes** (dynamic changepoints) to capture the underlying evolution of the buying behavior associated with the new product (as timing process is not stationary). This extended the basic changepoint framework by allowing the changepoint process itself to evolve over time. 

We shall show that dynamic changepoint model accurately tracks (and forecasts) the total sales curve as well as its trial and repeat components and other managerial diagnostics (e.g., percent of triers repeating).

## Imports

In [1]:
# Import Packages
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.special import gammaln

# Import Data
df = pd.read_csv('data/kiwibubbles/kiwibubbles_tran.csv')
df['Time'] = (df['Week'] - 1) * 7 + df['Day']
num_panelists = 2_799

## Introduction

For many new products, it is difficult to get an accurate read on their long-term potential based on only a few initial weeks of postlaunch (or test-market) sales data. Common problems include the following types of issues: (1) Significant promotional activity can artificially skew the initial sales levels, (2) early buyers may not exhibit typical purchasing rates, and (3) repeat-purchasing patterns may be hard to sort out from the voluminous amounts of first purchase (or trial) data. It is therefore essential for practitioners to rely on formal models of new product sales to tease apart and understand each of these underlying components to create a valid sales forecast.

At the heart of a new product sales-forecasting model is a multiple-event timing process. For many
behavioral processes besides new product sales forecasting, researchers need to capture a series of interpurchase cycles while accommodating customer heterogeneity. In addition, they need to filter out the influences that exogenous factors, such as promotional activities, may exert within and across these multiple purchase cycles.

## Models

### Stochastic Interpurchase Time Models Without Covariates

**Note**: 

- Log likelihood for $N$ consumers is simply the log likelihood of consumer $i$ summed over $i = 1$ to  $N$.
- $\lambda$ = purchase rate, $t_{ic}$ = censored time, $n_i$ = number of complete observations for consumer $i$, $t_{is}$ = sum of $n_i$ purchase times for consumer $i$.
- $r$ and $\alpha$ are parameters of the gamma mixing distribution.

#### Model: Exponential 

Log-Likelihood Function:

$$
LL = \sum^{N}_{i=1}n_i \log\lambda - \lambda(t_{is}+t_{ic})
$$

where:

$$
\begin{align*}
t_{is} &= \sum_{j=1}^{n_i} t_{ij} \\
&= \text{sum of } n_i \text{ interpurchase times of consumer } i
\end{align*}
$$

and 

$$
t_{ic} = \text{censored time for consumer }i
$$

#### Model: Exponential/Gamma

Log-Likelihood Function:

$$
LL = \sum_{i=1}^{N}\left[r \log \alpha + \sum_{j=0}^{n_i - 1} \log(r+j) - (n_i + r)\log(t_{is} + t_{ic} + \alpha)   \right]
$$

Identity from properties of the Gamma function:

$$
\sum_{j=0}^{n_i - 1} \log(r + j) = \log\left(\frac{\Gamma(r + n_i)}{\Gamma(r)}\right) = \texttt{gammaln}(r + n_i) - \texttt{gammaln}(r)
$$


In [2]:
r = 0.079
alpha = 71.375
calib = 182
df_calib = df[df['Time'] <= calib]
calib_buyers = df_calib['Panelist ID'].unique()

ll = 0
for i in range(num_panelists):
    if i < len(calib_buyers):
        panelist = calib_buyers[i]
        cust_data = df_calib[df_calib['Panelist ID'] == panelist].sort_values('Time')
        purchase_times = cust_data['Time'].to_numpy()
        n = len(cust_data)
        ts = np.sum(np.diff(purchase_times, prepend=0))
        tc = calib - purchase_times[-1]          
    else:
        n, ts = 0, 0
        tc = calib
    ll += r * np.log(alpha) + gammaln(r + n) - gammaln(r)
    ll -= (n + r) * np.log(ts + tc + alpha)

print(ll)

-3812.397650035847


In [3]:
r = 0.079
alpha = 71.375
calib = 182
df_calib = df[df['Time'] <= calib]

grouped = df_calib.sort_values(['Panelist ID', 'Time']).groupby('Panelist ID')

def summarize_customer(group):
    times = group['Time'].values
    n = len(times)
    t_is = np.sum(np.diff(times, prepend=0))
    t_ic = calib - times[-1]
    return pd.Series({'n': n, 't_is': t_is, 't_ic': t_ic})

summary = grouped.apply(summarize_customer, include_groups=False).reset_index()

summary = summary[summary['n'] > 0]

n = summary['n'].values
t_is = summary['t_is'].values
t_ic = summary['t_ic'].values

ll = np.sum(
    r * np.log(alpha)
    + gammaln(r + n) - gammaln(r)
    - ((n + r) * np.log(t_is + t_ic + alpha))
)

print(ll)

-3558.97758883386


$$
LL = \sum_{i=1}^{N} \left[r \log \alpha + \log A_{ipk} + \sum_{j=0}^{n_i - 1}\log(r+j)-(n_i + r)\log(B_{isk}+B_{ick}+\alpha)\right]
$$

$A_{ipk} = \prod_j A_{ijk} = \prod_j \exp(\beta\mathbf{X}_{ijk})$ where $\mathbf{X}$ are covariates, $B_{ijk}$

$B_{isk} = \sum_j B_{ijk}$

$B_{ipk} = \prod_j B_{ijk}$

$A_{k} = \exp(\beta\mathbf{X}_k)$

$B_{k} = A_1 d_1 + \sum_{w=2}^{k-1}A_{w} + A_{k}[t - d_1 - \gamma(k-2)]$

if there are no covariates, $\beta \mathbf{X} = 0$ and hence $A_{ijk} = 1$ and $B_{ijk} = t_{ij}$

#### Model: Erlang-2

Log-Likelihood Function:

$$
LL = \sum_{i=1}^{N} 2n_i \log \lambda + \log t_{ip} - \lambda (t_{is} + t_{ic}) + \log(1 + \lambda t_{ic})
$$

where:

$$
\begin{align*}
t_{ip} &= \prod_{j=1}^{n_i} t_{ij} \\
&= \text{product of interpurchase times for consumer }i
\end{align*}
$$

#### Model: Erlang-2/Gamma

Log-Likelihood Function:

$$
LL = \sum_{i=1}^{N} \left[  r \log\alpha + \log t_{ip} + \sum_{j=0}^{2n_i - 1} \log (r+j) + \log\left(1+\frac{(2n_i + r)t_{ic}}{t_{is} + t_{ic} + \alpha}\right) - (2n_i + r)\log(t_{is}+t_{ic} + \alpha) \right]
$$



In [4]:
r = 0.095
alpha = 33.094
calib = 182

df_calib = df[df['Time'] <= calib]

grouped = df_calib.sort_values(['Panelist ID', 'Time']).groupby('Panelist ID')

def summarize_customer(group):
    times = group['Time'].values
    n = len(times)
    if n < 2:
        return pd.Series({'n': n, 't_is': 0.0, 't_ic': 0.0, 'log_t_ip': 0})

    interpurchase_times = np.diff(times, prepend=0)
    log_t_ip = np.sum(np.log(interpurchase_times))
    t_is = np.sum(interpurchase_times)
    t_ic = calib - times[-1]

    return pd.Series({'n': n, 't_is': t_is, 't_ic': t_ic, 'log_t_ip': log_t_ip})

summary = grouped.apply(summarize_customer, include_groups=False).reset_index()

n = summary['n'].values
t_is = summary['t_is'].values
t_ic = summary['t_ic'].values
log_t_ip = summary['log_t_ip'].values

ll = np.sum(
    r * np.log(alpha)
    + log_t_ip
    + gammaln(r + 2 * n) - gammaln(r)
    + np.log(1 + ((2 * n + r) * t_ic) / (t_is + t_ic + alpha))
    - (2 * n + r) * np.log(t_is + t_ic + alpha)
)



print(ll)

-3832.886443482842


### Stochastic Interpurchase Time Models With Covariates

**Note**: 

- Log likelihood for $N$ consumers is simply the log likelihood of consumer $i$ summed over $i = 1$ to  $N$.
- $\lambda$ = purchase rate, $t_{ic}$ = censored time, $n_i$ = number of complete observations for consumer $i$, $t_{is}$ = sum of $n_i$ purchase times for consumer $i$.
- $r$ and $\alpha$ are parameters of the gamma mixing distribution.
- the subscript $k$ indicates the week in which the current purchase is made, given the week of last purchase is designated as week 1, $(k \ge 1)$.